# 深度学习在召回阶段的应用


工业界推荐系统的抽象模型如下所示. 分为两个阶段的主要原因是, 在大数据量下, 当前的端到端方案无法满足推荐服务的速度要求. 

- 召回阶段的目的是用快速, 较简单的模型来筛选可能的候选物品 (这一阶段的输入输出: 百万/千万 -> 几百)
- 排序阶段的目的是用更精确复杂的模型, 进一步对候选物品进行排序, 排序的分数可以是单纯的预估点击率, 也可以是融入业务需求的加权分数. (这一阶段的输入输出: 几百 -> 十几)

不同阶段的输入输出, 所用特征, 计算效率和精度的要求都不尽相同. 

本文主要讨论深度学习模型如何在召回 (Recall) 和排序 (Ranking) 阶段的应用.

<img src="images/dl_in_rec_sys.png" width="600"/>

## 传统召回阶段的实现

传统的多路召回, 每一路都基本关注一个单独维度, 或简单的模型召回. 例如

- 热榜召回 (历史 ctr)
- LBS 召回
- 兴趣画像召回
- Topic 召回
- ALS 召回 (矩阵分解)
- Item2Vec 召回
- ItemBased 协同召回

传统多路的优势在于, 开发难度不大; 可以互为补充; 整个多路结构服务可靠性较高. 

而缺点在于, 很难确定每一路召回选取多少条目合适(TopK 的K 值); 重复率不可控, 作为整体输出多少条目进入下一阶段也不可控; 且维护成本高.

## 深度学习应用

在召回阶段, 首先要考虑的还是速度和效率. 如果我们能得到用户和物品的 embedding 向量表达. 那么我们就可以通过向量之间的关系, 来计算用户和物品之间的关系.

**抽象的计算说明:**

- **这个阶段使用的深度模型, 基本上都是为了得到模型的中间产物, 即用户和物品的 embedding.** 
- 通过离线的模型计算, 得到向量表达之后, 在线上服务的时候, 可以通过快速的计算(例如 近似的相似度计算, 或利用 faiss)

<img src="images/dl_in_rec_recall_flow.png" width="600"/>

**在实际应用中, 通常也有两种实现方式:**

- 作为单独的一路召回
- 代替所有原始的召回策略, 通过对模型的输入控制, 在表示向量中融合进各种信息, 从而达到统一召回的能力

<img src="images/dl_in_rec_recall_mode.png" width="600"/>

## 具体方式

如何来学习用户和物品的 embedding 呢?

- 传统方法
    - 可以用 FM 来做
- 深度学习方法, 又可以分为以下几类
    - 无监督方式: Auto-Encoder
    - 半监督方式: Auto-Encoder + 矩阵分解
    - 监督方式
        - 针对 Rating 问题的模型
        - 针对 Ranking 问题的模型
        
我们先提出一个用户和物品的特征抽取框架, 也就是通过不同的特征抽取器, 从不同的输入中得到每个用户和每个物品的向量表达. 输入上, 我们一般可用的是, 用户对物品的评分矩阵, 用户对物品的隐式反馈, 常规属性, 多模态信息等. 多个抽取器通常会输出多个向量, 最终的表示方法通常是将这些向量进行拼接, 或平均.

<img src="images/dl_in_rec_recall_base_model.png" width="600"/>



### 无监督 Auto-Encoder 

#### I-AutoRec

是这一系列模型的鼻祖, 分为 I-AutoRec (Itembased AutoRec), U-AutoRec (Userbased AutoRec), 区别在于输入的不同. 以 I-AutoRec 为例, 输入是 user i, 对 m 个物品的评分 r

其重建目标为 h,

$$
h\left(\mathbf{r}^{(i)} ; \theta\right)=f\left(W \cdot g\left(V \cdot \mathbf{r}^{(i)}+\mu\right)+b\right)
$$

AutoRec 的目标函数如下, $\|\cdot\|_{\mathcal{O}}^{2}$ 表示模型只考虑被观察到的评分

$$
\underset{\theta}{\operatorname{argmin}} \sum_{i=1}^{N}\left\|\mathbf{r}^{(i)}-h\left(\mathbf{r}^{(i)} ; \theta\right)\right\|_{\mathcal{O}}^{2}+\lambda \cdot \text { reg }
$$

$f$ 和 $g$ 是两个激活函数, 可以灵活调整. 若有 n 个用户, m 个物品, 隐层神经元个数为 k, 那么从模型中我们可以得到用户对应隐层的表示为 $U_{em} \in R^{n\times k}$, 隐层到 h 的权重为 $I_{em}^T \in R^{m\times k}$. 那么用户和物品对应 embedding 向量的点积即为其预测评分.

论文中还指出了一些关键点:
- I-AutoRec 比 U-AutoRec 效果更好, 这个通常是因为, 相对于用户的数量来说, 物品的数量相对要少很多.
- 两个激活函数的选择, 会极大的影响效果
- 增加隐层的神经元个数, 也会对效果有提升
- 增加更多层, 更深的网络, 会有少量的效果提升

> **[2015] AutoRec: Autoencoders Meet Collaborative Filtering** <br>
> Suvash Sedhain, Aditya Krishna Menon, Scott Sanner, Lexing Xie Published in WWW 2015 DOI:10.1145/2740908.2742726 <br>
> http://users.cecs.anu.edu.au/~akmenon/papers/autorec/autorec-paper.pdf

<img src="images/dl_in_rec_recall_I_AutoRec.png" width="400"/>

#### **Collaborative Denoising Auto-Encoder(CDAE)**

CDAE 对于 AutoRec 最大的改进, 在于引入了 denosing 的技术. 提升模型鲁棒和泛化的能力. denosing 源于图像处理中 AutoEncoder 对随机输入像素的屏蔽. 对应到推荐场景下, 就是对随机评分的忽略. 可以类比作用与输入的 DropOut.


> **[2016] Collaborative denoising auto-encoders for top-n recommender systems** <br>
> Yao Wu, Christopher DuBois, Alice X. Zheng, Martin Ester Published in WSDM 2016 DOI:10.1145/2835776.2835837

<img src="images/dl_in_rec_recall_CDAE.png" width="400"/>

#### **Collaborative Fillering Neural Network**

CFN 是 AutoRec 的一种扩展, 它提出了两个方面的改进. (1)应用 denoising 的技术, 使得模型更加鲁棒, 泛化能力更好. (2)融入 side info, 比如用户画像, 物品描述, 来缓解稀疏性和冷启动影响.

> **[2015] Collaborative Filtering with Stacked Denoising AutoEncoders and Sparse Inputs** <br>
> https://hal.inria.fr/hal-01256422/document <br>
> <br>
> **[2016] Hybrid Recommender System based on Autoencoders** <br>
> Florian Strub, Jérémie Mary, Romaric GaudelPublished in DLRS@RecSys 2016 DOI:10.1145/2988450.2988456

### 半监督 Auto-Encoder + 矩阵分解

#### **Deep Collaborative Filltering Freamework**

这是一种通用框架, 实现了深度方法和协同模型的统一. 此框架能够融合深度信息(可以使用更多的 side info), 进而得到融合协同模型(hybrid collaborative models). 

$$
\underset{U, V}{\arg \min} \ell(R, U, V)+\beta\left(\|U\|_{F}^{2}+\|V\|_{F}^{2}\right)+\gamma \mathcal{L}(X, U)+\delta \mathcal{L}(Y, V)
$$

上面的 Loss Function, 描述了这个框架的思想, $X,Y$ 作为 side-info, $\ell(\cdot)$ 是协同过滤模型的 loss, $\mathcal{L}(X, U)$ 和 $\mathcal{L}(Y, V)$ 代表了从 side-info 得到的隐向量和协同过滤模型得到的隐向量的差别, 希望权衡这三部分, 使 loss 最小化.

> **[2015] Deep Collaborative Filtering via Marginalized Denoising Auto-encoder** <br>
> Sheng Li, Jaya Kawale, Yun FuPublished in CIKM 2015 DOI:10.1145/2806416.2806527

<img src="images/dl_in_rec_recall_semi-supervised.png" width="400"/>


还有很多论文可以看作是这个框架的特例, 例如

> [2013] Deep content-based music recommendation <br>
> Aäron van den Oord, Sander Dieleman, Benjamin SchrauwenPublished in NIPS 2013 <br>
> https://pdfs.semanticscholar.org/7145/44b7cf35a3b8bdc12fb1967624a38f257a42.pdf?_ga=2.169417830.653065060.1558323784-283354327.1540195118
> 
> 这篇论文使用的 ItemFeat + FM, 仅考虑了对音乐物品的深度信息的抽取 + 隐式反馈的 MF

> [2015] Collaborative Deep Learning for Recommender Systems <br>
> Hao Wang, Naiyan Wang, Dit-Yan YeungPublished in KDD 2015 DOI:10.1145/2783258.2783273

> [2014] Improving Content-based and Hybrid Music Recommendation using Deep Learning <br>
> Xinxi Wang, Ye WangPublished in ACM Multimedia 2014 DOI:10.1145/2647868.2654940


### 有监督之 Rating

在上文所述的特征抽取框架上, 加上不同的 Matching Function, 稍作改动就可以对 rating 或者 ranking 问题进行建模. rating 和 ranking 的区别, 从另一个角度也是两种目标函数的区别: 

- rating 对应 point-wise 目标, 通常是考虑回归框架, 来最小化预测值和目标值的平方误差, 通常把未观测样本当做负样本或采样后当做负样本.
- ranking 对应 pair-wise 目标, 则是考虑用户对观测到的样本的偏好大于未观测到的样本, 目标是最大化两两物品之间的偏好间隔.

对于Rating 问题, 最后的输出是一个由用户的向量和物品的向量通过 Matching Function 计算的 Score.

<img src="images/dl_in_rec_recall_base_model_rating.png" width="600"/>

#### **Neural Collaborative Filtering Framework**

对应上述的模型框架来看这篇论文提出的方法. 可以看到在用户和物品的表达上, 只用了一个 Embedding 层, 做了 dense 化的表示, 随后 Matching Function 的部分是用一个几层的全连接网络来实现的. 

然后考虑到隐式反馈的目标值是 {0,1}, 本文设定预测值 $\hat{y} \in [0,1] $, 为用户会喜欢物品的概率. 最终的 loss function 如下.

$$
L = -\sum_{(u, i) \in \mathcal{Y} \cup \mathcal{Y}^{-}} y_{u i} \log \hat{y}_{u i}+\left(1-y_{u i}\right) \log \left(1-\hat{y}_{u i}\right)
$$

> [2017] **Neural Collaborative Filtering** <br>
> Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu, Tat-Seng Chua Published in WWW 2017 DOI:10.1145/3038912.3052569 <br>
> https://arxiv.org/pdf/1708.05031.pdf

<img src="images/dl_in_rec_recall_NCFF.png" width="400"/>

#### **Youtube Video Recommendation**

本文中将推荐问题转化为一个极端庞大的分类问题, 类别数等于所有物品的数量, 每个物品作为一类, 对于给定的用户 U 和上下文 C, 我们要推荐一个物品 i, $i \in V$, 用 softmax 来计算每一个物品的可能性 $P(i|U,C)$. $u \in R^{N}$ 代表用户的 N 维向量表示, $v \in R^{N}$ 代表物品的 N 维向量表示.

$$
P\left(i | U,C\right)=\frac{e^{v_i u}}{\sum_{j \in V} e^{v_j u}}
$$

为了训练这样超大的分类问题, 本文使用了负采样(negative sampling) 的训练方式 (hierarchical softmax 的方法未能得到理想的结果). 从训练中, 我们就能得到用户和物品的向量表达 u, v. 在召回阶段的时候直接做内积或近似计算就能得到排名靠前的推荐物品了.

> **[2010] The YouTube video recommendation system** <br>
> James Davidson, Benjamin Liebald, Junning Liu, Palash Nandy, Taylor Van Vleet, Ullas Gargi, Sujoy Gupta, Yu He, Michael Ian Lambert, Blake Livingston, Dasarathi Sampath Published in RecSys 2010 DOI:10.1145/1864708.1864770

<img src="images/dl_in_rec_recall_youtube.png" width="600"/>

### 有监督之 Ranking

在之前的抽象模型上继续修改 Matching Function, 和训练样本的组织方式, 就可以变化为 pairwise 的 ranking 问题的解法. 

<img src="images/dl_in_rec_recall_base_model_raking.png" width="600"/>

#### **Facebook Product Recommendation**

这篇论文提出的 pairNN 模型, 是使用成对排序损失的协同深度模型 (Collaborative deep learning with pairwise rank loss). 

pairwise rank loss 来自于 Learn to Rank 问题的 Pairwise Approach. 它把排序问题转换成两个物品顺序的分类.

$$
\min _{\Theta} \sum_{(u \in U)} \sum_{(p_{+} \in P_{+})} \sum_{(p_{-} \in P_{-})} \max \left(0, r_{u p_{-}}-r_{u p_{+}}+\xi\right)
$$
$$
\min _{\Theta} \sum_{\left(u, p_{+}, p_{-}\right) \in \mathcal{E}} \max \left(0, r_{u p_{-}}-r_{u p_{+}}+\xi\right)
$$

原始日志中, 用户喜欢的物品是正例, 正例得分 $r_{u p_{+}}$ , 不喜欢的是负例, 负例得分 $r_{u p_{-}}$, 一个训练样本为三元组 $(u, p_+, p_-)$; 得分 $r_{u p_{+}}$ , $r_{u p_{-}}$ 则是通过两个模型分别学习用户和物品的向量表达后, 再用余弦相似度计算得到. $ξ$ 作为界限 (margin), 在论文中被设置为1.

那么分别对用户和物品建模的方式, 其实可以是非常灵活的, 论文中使用了 CNN, RNN, 包括预训练的 Word2Vec 等.

本文所用的 pairwise rank loss, 可以理解为 BPR Loss 的一个特例, 也是专门对于 ranking 问题进行建模, 可以只考量 AUC, 效果较好. 关于 BPR, 下文再述.


> **[2018] Collaborative Multi-modal deep learning for the personalized product retrieval in Facebook Marketplace** <br>
> Lu Zheng, Zhao Tan, Kun Han, Ren Mao Published in ArXiv 2018 <br>
> https://arxiv.org/pdf/1805.12312.pdf

<img src="images/dl_in_rec_recall_facebook.png" width="600"/>

#### **BPR (Bayesian Personalized Ranking Objective)**

对于 Ranking 问题, 更通用的一种目标函数是 Bayesian Personalized Ranking Objective. 首先通常的隐式反馈只有正反馈, 其余的有两种情况, 负反馈或者未知, 通常的方式是这两种情况都被当作是负反馈, 填充以0值. 但 BPR 提出的做法则是, 考虑物品两两之间的排序问题, 把原始的数据拆分得到新的训练数据集 $DS: U × I × I$, U 是用户数, I 是物品数. 如下图所示, 就是对 $u_1$ 有 $(u_1,i_2,j_1), (u_1,i_2,j_4),(u_1,i_3,j_1),(u_1,i_3,j_4)$

$(u, i, j) \in DS$ 代表的意思就是, 用户 u 在 物品 i 和 j 之间, 更偏好 i.

> [2009] BPR: Bayesian Personalized Ranking from Implicit Feedback <br>
> Steffen Rendle, Christoph Freudenthaler, Zeno Gantner, Lars Schmidt-Thieme Published in UAI 2009 <br>
> https://arxiv.org/pdf/1205.2618.pdf

<img src="images/dl_in_rec_recall_pairwise.png" width="600"/>

BPR 是基于贝叶斯思想的最大后验优化问题的, 如下

$>_{u}$ 代表我们考虑的样本, 为使后验最大化, 我们考虑它的似然 $p(>_{u} | \Theta )$ 和 $p(\Theta)$. 假设每个用户的偏好是独立的, 且每个用户偏好的物品之间也是没有独立的. 则似然表示成连乘的形式如下. $\delta(b)$ 是指示函数 (indicator function).

$$
\prod_{u \in U} p(>_{u} | \Theta)=\prod_{(u, i, j) \in U \times I \times I} p(i>_{u} j | \Theta)^{\delta((u, i, j) \in DS)} \cdot(1-p(i>_{u} j | \Theta))^{\delta((u, j, i) \notin DS)}
$$

$$
\delta(b) :=\left\{\begin{array}{ll}{1} & {\text { if } b \text { is true, }} \\ {0} & {\text { else }}\end{array}\right.
$$

进而推导出

$$
\prod_{u \in U} p\left(>_{u} | \Theta\right)=\prod_{(u, i, j) \in D_{S}} p\left(i>_{u} j | \Theta\right)
$$

这里我们用 $\hat{x}_{u i j}(\Theta)$ 表示从参数 $\Theta$ 得到的一个实数, 代表了用户 u 和 物品 i,j 的关系, 再将这个实数通过 sigmod 映射到 0-1, 作为一组 (u,i,j)的似然概率的表示.

$$
p\left(i>_{u} j | \Theta\right) :=\sigma\left(\hat{x}_{u i j}(\Theta)\right), \text{where } \sigma(x) :=\frac{1}{1+e^{-x}}
$$

解决了似然的部分, 对于参数的先验, 论文中假设, 参数符合高斯分布 $p(\Theta) \sim N\left(0, \Sigma_{\Theta}\right) $, 且为了减少参数量, 假设分布的协方差矩阵为对角矩阵 $\Sigma_{\Theta}=\lambda_{\Theta} I $. 因此先验的这部分也相当于最终损失函数的 L2 正则项.

综上, 完整的最大后验的优化问题表述如下:

$$
\begin{aligned} 
\text{BPR-OPT} & :=\ln p\left(\Theta |>_{u}\right) \\ 
&=\ln p\left(>_{u} | \Theta\right) p(\Theta) \\
&=\ln \prod_{(u, i, j) \in D_{S}} \sigma\left(\hat{x}_{u i j}\right) p(\Theta) \\ 
&=\sum_{(u, i, j) \in D_{S}} \ln \sigma\left(\hat{x}_{u i j}\right)+\ln p(\Theta) \\ 
&=\sum_{(u, i, j) \in D_{S}} \ln \sigma\left(\hat{x}_{u i j}\right)-\lambda_{\Theta}\|\Theta\|^{2} 
\end{aligned}
$$

因为本论文提出的只是 BPR loss function, 因此可以和原来的一些方法配合使用, 只需要替换原算法的 loss 部分.

## 小结

**建议**
- Auto-Encoder 方法可能不太实用, 原因在于很难重构许多多模态的 side info, 无法利用许多额外信息
- 有的方法只能使用 userId, itemId的数据, 不能容纳 side info 的方法是不实用的
- 论文水的很多, 要有辨别能力
- 参考公司发的 paper, 工程意义更大一些
- 尝试 BRP Loss 及相应框架

**有监督模型的改进点**
- 输入的变化, 用更丰富的 side info
- 不同的抽取器, 不同的网络
- 不同的 Matching Function

**可尝试的方向**
- 尝试应用 BPR 作为优化目标
- 使用深度模型学习 Embedding 表达, 进而召回

## Reference

- "深度学习在推荐系统应用的技术进展", 张俊林, 微博
- Deep Learning Based Recommender System: A Survey and New Perspectives ; Shuai Zhang, Lina Yao, Aixin SunPublished in ACM Comput. Surv. 2019 


- AutoRec: Autoencoders Meet Collaborative Filtering ; Suvash Sedhain, Aditya Krishna Menon, Scott Sanner, Lexing Xie Published in WWW 2015 
- Collaborative denoising auto-encoders for top-n recommender systems ; Yao Wu, Christopher DuBois, Alice X. Zheng, Martin Ester Published in WSDM 2016 
- Collaborative Filtering with Stacked Denoising AutoEncoders and Sparse Inputs
- Hybrid Recommender System based on Autoencoders ; Florian Strub, Jérémie Mary, Romaric GaudelPublished in DLRS@RecSys 2016 

- Deep Collaborative Filtering via Marginalized Denoising Auto-encoder ; Sheng Li, Jaya Kawale, Yun FuPublished in CIKM 2015

- Deep content-based music recommendation ; Aäron van den Oord, Sander Dieleman, Benjamin SchrauwenPublished in NIPS 2013 

- Collaborative Deep Learning for Recommender Systems ; Hao Wang, Naiyan Wang, Dit-Yan YeungPublished in KDD 2015

- Improving Content-based and Hybrid Music Recommendation using Deep Learning ; Xinxi Wang, Ye WangPublished in ACM Multimedia 2014

- Neural Collaborative Filtering ; Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu, Tat-Seng Chua Published in WWW 2017

- The YouTube video recommendation system ; James Davidson, Benjamin Liebald, Junning Liu, Palash Nandy, Taylor Van Vleet, Ullas Gargi, Sujoy Gupta, Yu He, Michael Ian Lambert, Blake Livingston, Dasarathi Sampath Published in RecSys 2010

- Collaborative Multi-modal deep learning for the personalized product retrieval in Facebook Marketplace ; Lu Zheng, Zhao Tan, Kun Han, Ren Mao Published in ArXiv 2018

- BPR: Bayesian Personalized Ranking from Implicit Feedback ; Steffen Rendle, Christoph Freudenthaler, Zeno Gantner, Lars Schmidt-Thieme Published in UAI 2009 

